In [51]:
# data analysis and wrangling
import pandas as pd
import numpy as np
import random as rnd

# visualization
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

# Pre processing 
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

# machine learning
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier

In [68]:
import os
#input_file = pd.ExcelFile(".\data\GPSRetention_v2.xlsx")
input_file = pd.ExcelFile(".\data\GPSRetention_Dummies.xlsx")
#GPSRetention = input_file.parse("Page1")
GPSRetention = input_file.parse("Sheet1")
print('Size of GPS Retention',GPSRetention.shape)
#output to csv file so we can read it later easier
GPSRetention.to_csv(".\data\GPSRetention_Dummies_v2.csv", header=True)

Size of GPS Retention (932, 84)


In [69]:
gps_df = pd.read_csv('./data/GPSRetention_Dummies_v2.csv')
#gps_df = pd.read_csv('./data/GPSRetention.csv', dtype={'MAJOR_DESC':str})

#Re-name possible y values
gps_df.rename(columns = {'Retained?':'Retained'}, inplace = True)

In [65]:
#print(gps_df.head())
#print(gps_df.tail())

In [23]:
#using sk learn to create the categorical values
from sklearn.preprocessing import LabelEncoder
lb_make = LabelEncoder()
#gps_df["status_code"]      = lb_make.fit_transform(gps_df["STATUS_DESC"])
gps_df["gender_code"]      = lb_make.fit_transform(gps_df["GENDER"])
gps_df["major_code"]       = lb_make.fit_transform(gps_df["MAJOR_DESC"])
gps_df["citizenship_code"] = lb_make.fit_transform(gps_df["CITIZENSHIP_DESC"])
gps_df["veteran_code"]     = lb_make.fit_transform(gps_df["VETERAN_CATEGORY_DESC"].astype(str))
gps_df["religion_code"]    = lb_make.fit_transform(gps_df["RELIGION_DESC"])
gps_df["marital_code"]     = lb_make.fit_transform(gps_df["MARITAL_STATUS_DESC"])
gps_df["primary_ethnicity_code"]     = lb_make.fit_transform(gps_df["PRIMARY_ETHNICITY_DESC"])
gps_df["state_code"]     = lb_make.fit_transform(gps_df["STATE_PROVINCE"])
gps_df["immigration_code"] = lb_make.fit_transform(gps_df["IMMIGRATION_STATUS"].astype(str))
#gps_df["nation_code"] = lb_make.fit_transform(gps_df["NATION"].astype(str))

#making flag columns
gps_df['has_passport'] = np.where(gps_df['NATION_OF_CITIZENSHIP_DESC'].isnull(), 0, 1)
gps_df['has_visa']     = np.where(gps_df['VISA_TYPE_DESC'].isnull(), 0, 1) 

In [45]:
#Create buckets for GPA
#making a gpa bucket before creating the code
gps_df['gpa_bucket'] = pd.cut(gps_df['GPA'], 4) #having 5 gave no value
gps_df["gpa_code"]   = lb_make.fit_transform(gps_df["gpa_bucket"])
print(gps_df[['gpa_bucket', 'retained']].groupby(['gpa_bucket'], as_index=False).mean().sort_values(by='gpa_bucket', ascending=True))

#making CLEANED_PREVIOUS_GPA into buckets
gps_df['previous_gpa_bucket'] = pd.cut(gps_df['CLEANED_PREVIOUS_GPA'], 25) #having 5 gave no value
gps_df["previous_gpa_code"]   = lb_make.fit_transform(gps_df["previous_gpa_bucket"])

      gpa_bucket  retained
0  (-0.004, 1.0]  0.103448
1     (1.0, 2.0]  0.142857
2     (2.0, 3.0]  0.533333
3     (3.0, 4.0]  0.830573


In [25]:
#drop columns we dont want
gps_df = gps_df.drop(['Fake_ID','CREDITS_EARNED','GPA','STATUS_DESC','PROGRAM_DESC','MAJOR_DESC',
                     'SECOND_MAJOR_DESC','OUTCOME_NUMBER','OUTCOME_AWARDED_IND','CREDITS_ATTEMPTED',
                      'CREDITS_EARNED','GPA','CITIZENSHIP_DESC','RELIGION_DESC','VETERAN_CATEGORY_DESC',
                      'GENDER','PRIMARY_ETHNICITY_DESC','MARITAL_STATUS_DESC','CITY','COUNTY_DESC',
                      'STATE_PROVINCE','POSTAL_CODE','NATION','PASSPORT_ISSUE_NATION_DESC','IMMIGRATION_STATUS',
                      'NATION_OF_BIRTH_DESC','NATION_OF_CITIZENSHIP_DESC','VISA_TYPE_DESC','CLEANED_PREVIOUS_GPA',
                      'STATUS_DESC','OUTCOME_NUMBER','OUTCOME_AWARDED_IND','NATION',
                      #created columns
                      'gpa_bucket','Unnamed: 0','previous_gpa_bucket','gpa_code',
                      #no sure on these yet
                      'GPS_Start_Term_','GPS_Last_Term_','Took_Non_GPS_Course_Within_Year?'
                    ], axis=1)

['retained' 'HasCIS' 'HasMathStats' 'gender_code' 'major_code'
 'citizenship_code' 'veteran_code' 'religion_code' 'marital_code'
 'primary_ethnicity_code' 'state_code' 'immigration_code' 'has_passport'
 'has_visa' 'previous_gpa_code']


In [70]:
print(gps_df.columns.values)

['Unnamed: 0' 'CLEANED_PREVIOUS_GPA' 'HasCIS' 'HasMathStats'
 'MAJOR_DESC_Information Technology' 'MAJOR_DESC_Software Engineering'
 'MAJOR_DESC_Software Management' 'MAJOR_DESC_Software Systems'
 'CITIZENSHIP_IND_Y' 'RELIGION_DESC_Buddhist'
 'RELIGION_DESC_Chinese Religions' 'RELIGION_DESC_Decline to Respond'
 'RELIGION_DESC_Episcopalian' 'RELIGION_DESC_Hindu' 'RELIGION_DESC_Jewish'
 'RELIGION_DESC_Lutheran (ELCA)' 'RELIGION_DESC_Lutheran (non-ELCA)'
 'RELIGION_DESC_Methodist' 'RELIGION_DESC_Muslim'
 'RELIGION_DESC_No Religious Affiliation'
 'RELIGION_DESC_Orthodox Christian' 'RELIGION_DESC_Other Christian'
 'RELIGION_DESC_Other Non-Christian' 'RELIGION_DESC_Presbyterian'
 'RELIGION_DESC_Roman Catholic' 'VETERAN_CATEGORY_DESC_NA'
 'VETERAN_CATEGORY_DESC_Other Protected Veteran Only' 'GENDER_M'
 'PRIMARY_ETHNICITY_DESC_Asian'
 'PRIMARY_ETHNICITY_DESC_Black or African American'
 'PRIMARY_ETHNICITY_DESC_Declined'
 'PRIMARY_ETHNICITY_DESC_Race/Ethnicity Unknown'
 'PRIMARY_ETHNICITY_DESC_T

In [55]:
gps_df = gps_df.drop(['Unnamed: 0'], axis=1)

In [71]:
#output describe of each column
gps_df.describe()


,Unnamed: 0,CLEANED_PREVIOUS_GPA,HasCIS,HasMathStats,MAJOR_DESC_Information Technology,MAJOR_DESC_Software Engineering,MAJOR_DESC_Software Management,MAJOR_DESC_Software Systems,CITIZENSHIP_IND_Y,RELIGION_DESC_Buddhist,...,VISA_TYPE_DESC_F1 student from other school,VISA_TYPE_DESC_Intracompany Transferee,VISA_TYPE_DESC_Legal Permanent Resident,VISA_TYPE_DESC_NA,VISA_TYPE_DESC_Pending immigrant,VISA_TYPE_DESC_Spouse or child of F1,VISA_TYPE_DESC_Temp worker in specialty OCC,VISA_TYPE_DESC_US Citizen,VISA_TYPE_DESC_Unknown,Retained
count,932.000000,932.000000,932.000000,932.000000,932.000000,932.000000,932.000000,932.000000,932.000000,932.000000,...,932.000000,932.000000,932.000000,932.000000,932.000000,932.000000,932.000000,932.000000,932.000000,932.000000
mean,465.500000,3.116991,0.303648,0.041845,0.157725,0.424893,0.122318,0.120172,0.542918,0.018240,...,0.010730,0.003219,0.063305,0.784335,0.008584,0.001073,0.051502,0.003219,0.002146,0.763948
std,269.189524,0.363399,0.460079,0.200343,0.364679,0.494592,0.327828,0.325337,0.498422,0.133891,...,0.103082,0.056674,0.243641,0.411504,0.092299,0.032756,0.221138,0.056674,0.046299,0.424882
min,0.000000,1.948333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,232.750000,2.940000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
50%,465.500000,3.113852,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,...,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
75%,698.250000,3.316537,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,...,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
max,931.000000,4.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


# Split up data between train and test

In [73]:
X_train, X_test, y_train, y_test = train_test_split(gps_df.drop('Retained', axis=1), gps_df.Retained, test_size=0.33, random_state=42)

#output the size of test and train data
print(X_train.shape, y_train.shape, X_test.shape)

(624, 84) (624,) (308, 84)


# Logistics Regression

## Confidence Score (ACC %)

In [74]:
log_reg = LogisticRegression()
log_reg.fit(X_train, y_train)
y_pred = log_reg.predict(X_test)
acc_log = round(log_reg.score(X_train, y_train)*100, 2)
acc_log

79.650000000000006

## Column Coeffs

In [75]:
coeff_df = pd.DataFrame(gps_df.columns.delete(0)) #grabbing only column names into dataframe
coeff_df.columns = ['Feature'] #giving column a name
coeff_df["Correleation"] = pd.Series(log_reg.coef_[0]) #grabbing coeff of each column
coeff_df.sort_values(by='Correleation',ascending=False) #output coeff values

,Feature,Correleation
68,IMMIGRATION_STATUS_PR,1.513868
77,VISA_TYPE_DESC_NA,0.966050
4,MAJOR_DESC_Software Engineering,0.831571
33,MARITAL_STATUS_DESC_Legally Separated,0.760281
25,VETERAN_CATEGORY_DESC_Other Protected Veteran ...,0.705912
3,MAJOR_DESC_Information Technology,0.589579
56,STATE_PROVINCE_OH,0.581871
13,RELIGION_DESC_Jewish,0.579901
70,VISA_TYPE_DESC_Asylee,0.501781
38,STATE_PROVINCE_CA,0.484751


# Support Vector Machine

In [76]:
svc = SVC()
svc.fit(X_train, y_train)
y_pred = svc.predict(X_test)
acc_svc = round(svc.score(X_train, y_train)*100, 2)
acc_svc #outputting the acc score from SVM model

78.040000000000006

# K-Nearest Neighbors

In [77]:
knn = KNeighborsClassifier(n_neighbors = 3)
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
acc_knn = round(knn.score(X_train, y_train)*100, 2)
acc_knn

83.489999999999995

# Guassian Naive Bays

In [78]:
gaussian = GaussianNB()
gaussian.fit(X_train, y_train)
y_pred = gaussian.predict(X_test)
acc_gaussian = round(gaussian.score(X_train, y_train)*100,2)
acc_gaussian

31.57

# Perceptron

In [79]:
perceptron = Perceptron()
perceptron.fit(X_train, y_train)
y_pred = perceptron.predict(X_test)
acc_perceptron = round(perceptron.score(X_train, y_train)*100,2)
acc_perceptron

23.719999999999999

# Linear SVC

In [80]:
linear_svc = LinearSVC()
linear_svc.fit(X_train, y_train)
acc_linear_svc = round(linear_svc.score(X_train, y_train)*100, 2)
acc_linear_svc


63.299999999999997

# Stochastic Gradient Descent

In [81]:
sgd = SGDClassifier()
sgd.fit(X_train, y_train)
y_pred = sgd.predict(X_test)
acc_sgd = round(sgd.score(X_train, y_train)*100, 2)
acc_sgd

75.959999999999994

# Decision Tree

In [82]:
decision_tree = DecisionTreeClassifier()
decision_tree.fit(X_train, y_train)
y_pred = decision_tree.predict(X_test)
acc_decision_tree = round(decision_tree.score(X_train, y_train)*100,2)
acc_decision_tree

100.0

# Random Forest

In [83]:
random_forest = RandomForestClassifier(n_estimators=100)
random_forest.fit(X_train, y_train)
y_pred = random_forest.predict(X_test)
random_forest.score(X_train, y_train)
acc_random_forest = round(random_forest.score(X_train, y_train)*100, 2)
acc_random_forest

100.0

# Model Evaluation

In [84]:
models = pd.DataFrame({
    'Model': ['Support Vector Machines', 'KNN', 'Logistic Regression', 
              'Random Forest', 'Naive Bayes', 'Perceptron', 
              'Stochastic Gradient Decent', 'Linear SVC', 
              'Decision Tree'],
    'Score': [acc_svc, acc_knn, acc_log, 
              acc_random_forest, acc_gaussian, acc_perceptron, 
              acc_sgd, acc_linear_svc, acc_decision_tree]})
models.sort_values(by='Score', ascending=False)

,Model,Score
3,Random Forest,100.00
8,Decision Tree,100.00
1,KNN,83.49
2,Logistic Regression,79.65
0,Support Vector Machines,78.04
6,Stochastic Gradient Decent,75.96
7,Linear SVC,63.30
4,Naive Bayes,31.57
5,Perceptron,23.72
